In [ ]:
import os
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import display

%matplotlib inline

DIR = r'c://downloads'

plt.style.use('ggplot')

# Correlation

In [ ]:
# Code from last lesson - merging ViralZone and VIPERdb datasets

viperdb = pd.read_csv("viperdb.csv")
viralzone = pd.read_csv(os.path.join(DIR, 'viralzone.csv'))

viralzone['Genome'] = viralzone['Genome'].apply(json.loads)
viralzone['Genome Length'] = viralzone['Genome'].apply(lambda genome: sum([fragment['size'] for fragment in genome]))
viralzone_genome_length_per_genus = viralzone.groupby(['Group', 'Family', 'Genus'])['Genome Length'].max().reset_index()

viperdb_radiuses_per_genera = viperdb.groupby(['Family', 'Genus'])[['Inner Radius', 'Outer Radius']].max().reset_index()

combined_genera = pd.merge(viralzone_genome_length_per_genus, viperdb_radiuses_per_genera, on = ['Family', 'Genus'])
combined_genera['Log10 Genome Length'] = np.log10(combined_genera['Genome Length'])

print('We have %d combined genera.' % len(combined_genera))
display(combined_genera.head())

fig, ax = plt.subplots(figsize = (12, 6))
combined_genera.plot(ax = ax, kind = 'scatter', x = 'Log10 Genome Length', y = 'Inner Radius', s = 50)

In [ ]:
from scipy.stats import pearsonr, spearmanr

# Only barely significant
print('Pearson correlation between Genome Length and Inner Radius: r = %.2f, p-value = %.4f' % \
        pearsonr(combined_genera['Genome Length'], combined_genera['Inner Radius']))

In [ ]:
# What if we get rid of the two outliers?

filtered_genera = combined_genera[combined_genera['Log10 Genome Length'] < 5]
print('Filtered out %d outliers.' % (len(combined_genera) - len(filtered_genera)))

# Now it's really significant!
print('Pearson correlation between Genome Length and Inner Radius without outliers: r = %.2f, p-value = %e' % \
        pearsonr(filtered_genera['Genome Length'], filtered_genera['Inner Radius']))
        
# Was it justified to get rid of the outliers?
# Hard to tell...

In [ ]:
# Remember that the graph shows Log10 Genome Length, and not Genome Length!
# How would it affect the results?

print('Pearson correlation between Log10 Genome Length and Inner Radius: r = %.2f, p-value = %e' % \
        pearsonr(combined_genera['Log10 Genome Length'], combined_genera['Inner Radius']))
print('Pearson correlation between Log10 Genome Length and Inner Radius without outliers: r = %.2f, p-value = %e' % \
        pearsonr(filtered_genera['Log10 Genome Length'], filtered_genera['Inner Radius']))
        
# It strongly improves the original dataset, but gives similar results for the filtered one.

In [ ]:
# There aren't good reasons to assume linear relationship for either the raw length or its log transformation.
# Spearman's rank correlation might be more appropriate here.

print('Spearman correlation between Genome Length and Inner Radius: r = %.2f, p-value = %e' % \
        spearmanr(combined_genera['Genome Length'], combined_genera['Inner Radius']))
print('Spearman correlation between Genome Length and Inner Radius without outliers: r = %.2f, p-value = %e' % \
        spearmanr(filtered_genera['Genome Length'], filtered_genera['Inner Radius']))
print('Spearman correlation between Log10 Genome Length and Inner Radius: r = %.2f, p-value = %e' % \
        spearmanr(combined_genera['Log10 Genome Length'], combined_genera['Inner Radius']))
print('Spearman correlation between Log10 Genome Length and Inner Radius without outliers: r = %.2f, p-value = %e' % \
        spearmanr(filtered_genera['Log10 Genome Length'], filtered_genera['Inner Radius']))
        
# As anticipated, the log transformation has no effect at all.
# Also the two "outliers" don't play any significant role.

**Conclusion**: The genome length of viruses indeed seems correlated to their inner capsid size (not very surprisingly...)

In [ ]:
# Pandas offers a shortcut for calculating all-vs-all correlations

print('Pearson correlations:')
display(combined_genera.corr())

print('Spearman correlations:')
display(combined_genera.corr(method = 'spearman'))

# Linear regression

In [ ]:
from statsmodels import api as sm

y = combined_genera['Inner Radius']
x = sm.add_constant(combined_genera['Log10 Genome Length']) # Important to add_constant. It is not added by default.
regression_model = sm.OLS(y, x)
regression_results = regression_model.fit()
print(regression_results.summary())

# As expected, we get the same results.

In [ ]:
# But now we have a concrete model in hand.
print(regression_results.params)
type(regression_results.params)

In [ ]:
# Let's plot it.

fig, ax = plt.subplots(figsize = (12, 6))

regression_x = np.array([3.0, 6.5])
regression_y = regression_results.predict(sm.add_constant(regression_x))
ax.plot(regression_x, regression_y, '--')

combined_genera.plot(ax = ax, kind = 'scatter', x = 'Log10 Genome Length', y = 'Inner Radius', s = 50)

ax.set_xlim((3.0, 6.5))

# Comparing two populations

In [ ]:
viralzone_rna = viralzone[viralzone['Group'].str.contains('RNA')]
viralzone_dna = viralzone[viralzone['Group'].str.contains('DNA')]

fig, ax = plt.subplots(figsize = (12, 6))
ax.boxplot([np.log10(viralzone_rna['Genome Length']), np.log10(viralzone_dna['Genome Length'])], showmeans = True)
# e.g. no log10
# ax.boxplot([viralzone_rna['Genome Length'], viralzone_dna['Genome Length']], showmeans = True)
ax.set_xticklabels(['RNA Viruses (%d)' % len(viralzone_rna), 'DNA Viruses (%d)' % len(viralzone_dna)], fontsize = 13)
ax.set_ylabel('Log10 Genome Length')
# DNA viruses are orders of magnitude different in length (on both sides)
# which is why we use the log10, allowing us to show it better in a single graph

In [ ]:
from scipy.stats import ttest_ind, mannwhitneyu

t_stat, t_pval = ttest_ind(viralzone_rna['Genome Length'], viralzone_dna['Genome Length'], equal_var = False)
u_test, u_pval = mannwhitneyu(viralzone_rna['Genome Length'], viralzone_dna['Genome Length'], alternative = 'two-sided')
print('Significance: %.2g (t-test), %.2g (U-test)' % (t_pval, u_pval))
# T-test gives more significant results; U-test is more conservative
# Either way, we can conclude that the means are very different

What happens if we normalize the two groups and force them to have the same mean (0) and variance (1)?

In [ ]:
# converting data to Z-values
def normalize(values):
    return (values - np.mean(values)) / np.std(values)

normalized_rna_genome_lengths = normalize(viralzone_rna['Genome Length'])
normalized_dna_genome_lengths = normalize(viralzone_dna['Genome Length'])

fig, ax = plt.subplots(figsize = (12, 6))
ax.boxplot([normalized_rna_genome_lengths, normalized_dna_genome_lengths], showmeans = True)
ax.set_xticklabels(['RNA Viruses (%d)' % len(viralzone_rna), 'DNA Viruses (%d)' % len(viralzone_dna)], fontsize = 13)
ax.set_ylabel('Normalized Genome Length')

In [ ]:
_, t_pval = ttest_ind(normalized_rna_genome_lengths, normalized_dna_genome_lengths, equal_var = False)
_, u_pval = mannwhitneyu(normalized_rna_genome_lengths, normalized_dna_genome_lengths, alternative = 'two-sided')
print('Significance: %.2g (t-test), %.2g (U-test)' % (t_pval, u_pval))

T-test tests for mean, so it is expected to get 1 as a result, since Z-scaling normalizes the means of both samples.
U-test can give p<0.05 even if the mean is the same (in theory), but in this case it doesn't.

However, even if the mean is the same, we can still test whether the distribution of the data is different using KS test.

In [ ]:
from scipy.stats import ks_2samp

_, ks_pval = ks_2samp(normalized_rna_genome_lengths, normalized_dna_genome_lengths)
print('Kolmogorov-Smirnov significance: %.2g' % ks_pval)

# Contingency tables

In [ ]:
cancer_data = pd.DataFrame([[286, 227], [114, 373]], columns = ['Lung cancer', 'Healthy'], index = ['Smokers', 'Non-smokers'])
display(cancer_data)

In [ ]:
from scipy.stats import fisher_exact, chi2_contingency

# Fisher's exact test
print('OR = %.2f, p-value = %e' % fisher_exact(cancer_data))

# Manual calculation of Fisher's exact test (FYI)
print('Self calculation of OR: %.2f' % ((cancer_data.loc['Smokers', 'Lung cancer'] / cancer_data.loc['Smokers', 'Healthy']) / \
        (cancer_data.loc['Non-smokers', 'Lung cancer'] / cancer_data.loc['Non-smokers', 'Healthy'])))
print()

# Chi squared test
chi2_statistic, pval, dof, expected_data = chi2_contingency(cancer_data)
print('Chi2 p-value = %e' % pval)
print()
print('Expected table:')
display(pd.DataFrame(expected_data, columns = cancer_data.columns, index = cancer_data.index))
print('The observed one:')
display(cancer_data)

In [ ]:
# From: ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_32/gencode.v32.annotation.gtf.gz
# You can analyze the code later
gene_annotations = pd.read_csv(os.path.join(DIR, 'gencode.v32.annotation.gtf.gz'), sep = '\t', comment = '#', \
        names = ['chr', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'extra_fields'])
        
def parse_extra_fields(raw_extra_fields):

    extra_fields = {}

    for raw_extra_field in raw_extra_fields[:-1].split(';'):
        key, raw_value = raw_extra_field.strip().split(' ')
        value = raw_value.strip('"')
        extra_fields[key] = value
        
    return extra_fields
        
gene_annotations['extra_fields'] = gene_annotations['extra_fields'].apply(parse_extra_fields)

genes = gene_annotations[gene_annotations['type'] == 'gene'].copy()
genes['gene_type'] = genes['extra_fields'].apply(lambda extra_fields: extra_fields['gene_type']) 

gene_types_per_chromosome = genes.groupby(['chr', 'gene_type']).size().unstack()[['protein_coding', 'miRNA']]\
        .fillna(0).astype(int)

# remove the labels for column and row indeces
gene_types_per_chromosome.columns.name = None
gene_types_per_chromosome.index.name = None

display(gene_types_per_chromosome)

In [ ]:
_, pval, _, expected_data = chi2_contingency(gene_types_per_chromosome)
print('%.2f of the expected values are above 5, %.2f are above 1' % (np.mean(expected_data >= 5), np.mean(expected_data >= 1)))
print('p-value = %e' % pval)

Different chromosomes have a tendency toward different gene types, but we don't know which are the chromosomes with a significantly unexpected tendency.
__It's important not to confuse significance with strength/effect size!__

# Permutation tests

In [ ]:
# Null hypothesis: (random) mutations distribute uniformly
# Alternative: they do not (i.e. there is a region of at least 300bps that is "protected" against mutations)
# What will be our p value? 5%?

GENE_SIZE = 1000
N_MUTATIONS = 15
OBSERVED_GAP_SIZE = 300

N_PERMUTATIONS = 5000

count = 0
sampled_max_gaps = []

for _ in range(N_PERMUTATIONS):
    
    # not the most efficient way to do this, but we are not running a large number of simulations
    sampled_mutation_locations = np.random.choice(np.arange(GENE_SIZE), N_MUTATIONS, replace = False)
    # sort random mutations between the start and the end of the gene
    # calculate gaps between them using np.diff
    gaps = np.diff(np.concatenate([[0], np.sort(sampled_mutation_locations), [GENE_SIZE - 1]]))
    max_gap = np.max(gaps) # select the largest gap
    
    # see whether the simulated result is equal to or more extreme than the result we've observed
    if max_gap >= OBSERVED_GAP_SIZE:
        count += 1
        
    sampled_max_gaps.append(max_gap)

# in this case we most probably only need one-tailed since we are looking only for "larger than (or equal)"
# however, since we have to be objective and sceptical, it is generally more appropriate to use two-tailed pval 
one_tailed_pval = count / N_PERMUTATIONS
two_tailed_pval = 2 * one_tailed_pval
print('Estimated p-value - one-tailed: %.3f, two-tailed: %.3f' % (one_tailed_pval, two_tailed_pval))

In [ ]:
fig, ax = plt.subplots()
ax.hist(sampled_max_gaps, bins = 30, color = 'teal')
ax.axvline(300, linestyle = '--')

### The Downsides

Can be very computationally expensive for accurate estimation of low p-values (impractical for very low ones)

Results may change every time you run the test

# Multiple testing

In [ ]:
total_genes = gene_types_per_chromosome.sum()
print(total_genes)
print('Total miRNA frequency: %.3f' % (total_genes['miRNA'] / total_genes.sum()))

def test_chromosome_genes(chr_genes):
    
    other_genes = total_genes - chr_genes
    
    chr_miRNA_freq = chr_genes['miRNA'] / chr_genes.sum()
    other_miRNA_freq = other_genes['miRNA'] / other_genes.sum()
    RR = chr_miRNA_freq / other_miRNA_freq # We can actually compute RR, not just OR
    
    contingency_table = pd.DataFrame.from_records([chr_genes, other_genes], index = [chr_genes.name, 'others'])
    _, pval = fisher_exact(contingency_table)
    
    return pd.Series({'miRNA_freq': chr_miRNA_freq, 'RR': RR, 'pval': pval})

gene_types_per_chromosome = pd.concat([gene_types_per_chromosome, gene_types_per_chromosome.apply(test_chromosome_genes, \
        axis = 1)], axis = 1).sort_values('pval')
display(gene_types_per_chromosome)

In [ ]:
from statsmodels.stats.multitest import multipletests

gene_types_per_chromosome['FDR_significance'], gene_types_per_chromosome['FDR_qval'], _, _ = \
        multipletests(gene_types_per_chromosome['pval'], method = 'fdr_bh')
gene_types_per_chromosome['Bonferroni_significance'], gene_types_per_chromosome['Bonferroni_qval'], _, _ = \
        multipletests(gene_types_per_chromosome['pval'], method = 'bonferroni')

display(gene_types_per_chromosome)

In [ ]:
genome_lengths_per_group = viralzone_genome_length_per_genus.groupby('Group')['Genome Length']
group_names, group_lengths = zip(*genome_lengths_per_group)
group_sizes = list(map(len, group_lengths))

fig, ax = plt.subplots(figsize = (12, 6))
ax.boxplot(list(map(np.log10, group_lengths)), showmeans = True)
ax.set_xticklabels(['%s (%d)' % group_name_and_size for group_name_and_size in zip(group_names, group_sizes)])
ax.set_ylabel('Log10 Genome Length')

In [ ]:
global_genome_length_avg = viralzone_genome_length_per_genus['Genome Length'].mean()
print('Gloabal genome length average: %d' % global_genome_length_avg)

group_genome_lengths = pd.DataFrame({'#': group_sizes, 'avg': list(map(np.average, group_lengths)), \
        'std': list(map(np.std, group_lengths))}, index = group_names)
group_genome_lengths['avg_ratio'] = group_genome_lengths['avg'] / global_genome_length_avg

def calc_pval(group_name, lengths):
    other_mask = (viralzone_genome_length_per_genus['Group'] != group_name)
    other_lengths = viralzone_genome_length_per_genus.loc[other_mask, 'Genome Length']
    _, pval = ttest_ind(lengths, other_lengths, equal_var = False)
    return pval

group_genome_lengths['pval'] = [calc_pval(group_name, lengths) for group_name, lengths in genome_lengths_per_group]
group_genome_lengths.sort_values('pval', inplace = True)

group_genome_lengths['Bonferroni_significance'], group_genome_lengths['Bonferroni_qval'], _, _ = \
        multipletests(group_genome_lengths['pval'], method = 'bonferroni')

display(group_genome_lengths)